<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

In [107]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)
print (eiban)





LC7200322800994985921


###Payment Generator###

In [12]:
# Erstellung von Transaktionsdaten 
# Daten Payment ID z.B. 1059295797

from datetime import datetime, timedelta
import random
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 


# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)


# Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Zufallsauswahl Kundendaten mittels Vetragsnummer
cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))


# Erstellung der Zahlungs-Basisinformationen 

#start_date = datetime(2018,1,1,8,0,0)
trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'valuta_date','cust_vertrag_nr','cust_konto_nr',
                                 'cust_vname','cust_nname','cust_plz','cust_ort',
                                'cust_kanton','cust_sprachregion','cust_auth_device'])

for i in range(30):
    trx_data.loc[i,['timestamp']] = timestamps = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(seconds=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']])
    timestamps = datetime.strptime(timestamps,"%Y.%m.%d %H:%M")
    trx_data.loc[i,['valuta_date']] = valuta = timestamps + timedelta(hours=12*i)
    trx_data.loc[i,['cust_vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
    trx_data.loc[i,['cust_konto_nr']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']])
    trx_data.loc[i,['cust_vname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['vorname']])
    trx_data.loc[i,['cust_nname']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['nachname']])
    trx_data.loc[i,['cust_plz']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['plz']])
    trx_data.loc[i,['cust_ort']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['ort']])
    trx_data.loc[i,['cust_kanton']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['kanton']])
    trx_data.loc[i,['cust_sprachregion']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['sprachregion']])
    trx_data.loc[i,['cust_auth_device']] = np.ravel(cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['auth_device']])
    
trx_data.head(30)




,timestamp,paym_id,send_bc_nr,amount,rcv_bc_nr,rcv_bc_iban,rcv_bc_country,rcv_bc_code,valuta_date,cust_vertrag_nr,cust_konto_nr,cust_vname,cust_nname,cust_plz,cust_ort,cust_kanton,cust_sprachregion,cust_auth_device
0,2018.01.01 08:00,1191542592392,CHBANK71XXX,29081,ADBANK12XXX,AD1000060004451247870930,Andorra,AD,2018-01-01 08:00:00,78552,CH5300772546292429215,Silvia,Kretz,6102,Malters,LU,de,mtan
1,2018.01.01 08:00,1164767517735,CHBANK71XXX,25426,LCBANK66XXX,LC55HEMM000100010012001200023015,Saint Lucia,LC,2018-01-01 20:00:00,81857,CH5300772209299545512,Michael,Gerosa,6212,St. Erhard,LU,de,ptan
2,2018.01.01 08:01,1007985734858,CHBANK71XXX,67361,ITBANK39XXX,IT43K0310412701000000820420,Italien,IT,2018-01-02 08:01:00,75265,CH5300772068564924911,Peter,Schmidli,6114,Steinhuserberg,LU,de,ptan
3,2018.01.01 08:01,1070647811802,CHBANK71XXX,48221,SEBANK70XXX,SE0230000000030301099952,Schweden,SE,2018-01-02 20:01:00,79342,CH5300772814028946561,Laura,Albisser,6130,Willisau,LU,de,ptan
4,2018.01.01 08:02,1709346073956,CHBANK71XXX,97674,KWBANK45XXX,KW02NBOK0000000000001000614589,Kuwait,KW,2018-01-03 08:02:00,65326,CH5300772383558767996,Daniel,Menghini,6294,Ermensee,LU,de,mtan
5,2018.01.01 08:02,1498503998832,CHBANK71XXX,23144,GRBANK31XXX,GR0201102160000021661309175,Griechenland,GR,2018-01-03 20:02:00,70703,CH5300772244012441341,Philippe,Grosjean,1290,Versoix,GE,fr,ptan
6,2018.01.01 08:03,1416083080997,CHBANK71XXX,68235,IQBANK35XXX,IQ20CBIQ861800101010500,Irak,IQ,2018-01-04 08:03:00,76678,CH5300772305020920950,Alice,Niederberger,6162,Entlebuch,LU,de,mtan
7,2018.01.01 08:03,1442975262198,CHBANK71XXX,83979,ITBANK39XXX,IT43K0310412701000000820420,Italien,IT,2018-01-04 20:03:00,76890,CH5300772005883239437,Walter,Wenger,5000,Aarau,AG,de,mtan
8,2018.01.01 08:04,1302298327674,CHBANK71XXX,15643,IEBANK36XXX,IE02BOFI90008413113207,Irland,IE,2018-01-05 08:04:00,84474,CH5300772126799505302,Hans,Wiesner,6010,Kriens,LU,de,ptan
9,2018.01.01 08:04,1297688592228,CHBANK71XXX,70180,BRBANK17XXX,BR0200000000010670000117668C1,Brasilien,BR,2018-01-05 20:04:00,83234,CH5300772409401453188,Philippe,Blanc,1252,Meinier,GE,fr,ptan


In [149]:


import pandas as pd
from random import randint

df = pd.DataFrame(columns=['A','B','C','D','E'])

for i in range(5): #add 5 rows of data
    df.loc[i, ['A']] = randint(0,99)
    df.loc[i, ['B']] = randint(0,99)
    df.loc[i, ['C']] = randint(0,99)
    df.loc[i, ['D']] = randint(0,99)
    df.loc[i, ['E']] = randint(0,99)
    
df

,A,B,C,D,E
0,82.0,93.0,1.0,5.0,8.0
1,4.0,34.0,45.0,13.0,88.0
2,87.0,41.0,54.0,41.0,43.0
3,81.0,44.0,98.0,3.0,83.0
4,47.0,52.0,20.0,45.0,60.0
